In [ ]:
import sqlite3
import json
import os
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect(r"C:\Users\Administrator\Desktop\Miniproj\phonepe_database.db")

# Create a cursor object
cursor = conn.cursor()

# Create the Aggregated Transaction Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS aggregated_transaction (
        State TEXT,
        Year INTEGER,
        Quater INTEGER,
        Transacion_type TEXT,
        Transacion_count INTEGER,
        Transacion_amount REAL
    );
''')

# Commit the changes
conn.commit()

# Path to the JSON files


path = r"C:\Users\Administrator\Desktop\Miniproj\venv\pulse\data\aggregated\transaction\country\india\state"

# Extract data from JSON files and insert into the table
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for quarter in os.listdir(year_path):
            quarter_path = os.path.join(year_path, quarter)
            with open(quarter_path, 'r') as f:
                data = json.load(f)
                for transaction in data['data']['transactionData']:
                    cursor.execute('''
                        INSERT INTO aggregated_transaction (State, Year, Quater, Transacion_type, Transacion_count, Transacion_amount)
                        VALUES (?, ?, ?, ?, ?, ?);
                    ''', (
                        state,
                        int(year),
                        int(quarter.split('.')[0]),
                        transaction['name'],
                        transaction['paymentInstruments'][0]['count'],
                        transaction['paymentInstruments'][0]['amount']
                    ))

# Commit the changes
conn.commit()

# Close the connection
conn.close()
